In [1]:
#%cd /content/drive/MyDrive/Colab Notebooks
#!pip install pyspark


/content/drive/MyDrive/Colab Notebooks


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnull

spark = SparkSession.builder.appName("RodentInfestation").getOrCreate()

In [11]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Rodent_Inspection_20250504.csv', header=True, inferSchema=False)


In [12]:
#df.printSchema()
#df.show(5)
#df.describe().show()
#df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()


root
 |-- INSPECTION_TYPE: string (nullable = true)
 |-- JOB_TICKET_OR_WORK_ORDER_ID: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- JOB_PROGRESS: string (nullable = true)
 |-- BBL: string (nullable = true)
 |-- BORO_CODE: string (nullable = true)
 |-- BLOCK: string (nullable = true)
 |-- LOT: string (nullable = true)
 |-- HOUSE_NUMBER: string (nullable = true)
 |-- STREET_NAME: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- X_COORD: string (nullable = true)
 |-- Y_COORD: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- INSPECTION_DATE: string (nullable = true)
 |-- RESULT: string (nullable = true)
 |-- APPROVED_DATE: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- COMMUNITY BOARD: string (nullable = true)
 |-- COUNCIL DISTRICT: string (nullable = true)
 |-- CENSUS TRACT: string (nullable = true)
 |-- BIN: string (nullabl

In [13]:
from pyspark.sql.functions import to_timestamp

#df = df.filter(df["Inspection_Date"].isNotNull())

df = df.filter(
    (df["Inspection_Date"].isNotNull()) &
    (df["BOROUGH"].isNotNull()) &
    (df["LATITUDE"].isNotNull()) &
    (df["LONGITUDE"].isNotNull()) &
    (df["LATITUDE"] != 0) &
    (df["LONGITUDE"] != 0) &
    (df["RESULT"].isNotNull())
)

df = df.withColumn("Inspection_Date", to_timestamp("Inspection_Date", 'MM/dd/yyyy hh:mm:ss a'))

#df.printSchema()

#df.select("Inspection_Date").show(10)


root
 |-- INSPECTION_TYPE: string (nullable = true)
 |-- JOB_TICKET_OR_WORK_ORDER_ID: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- JOB_PROGRESS: string (nullable = true)
 |-- BBL: string (nullable = true)
 |-- BORO_CODE: string (nullable = true)
 |-- BLOCK: string (nullable = true)
 |-- LOT: string (nullable = true)
 |-- HOUSE_NUMBER: string (nullable = true)
 |-- STREET_NAME: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- X_COORD: string (nullable = true)
 |-- Y_COORD: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- Inspection_Date: timestamp (nullable = true)
 |-- RESULT: string (nullable = true)
 |-- APPROVED_DATE: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- COMMUNITY BOARD: string (nullable = true)
 |-- COUNCIL DISTRICT: string (nullable = true)
 |-- CENSUS TRACT: string (nullable = true)
 |-- BIN: string (null

In [14]:
#df.groupBy("Borough").count().show()


+-------------+------+
|      Borough| count|
+-------------+------+
|       Queens|166196|
|     Brooklyn|537746|
|Staten Island| 48396|
|    Manhattan|631234|
|        Bronx|538875|
+-------------+------+



In [15]:
from pyspark.sql.functions import year, month


# from pyspark.sql.functions import to_date

# df = df.withColumn("Inspection_Date", to_date("Inspection_Date", 'MM/dd/yyyy'))

df = df.withColumn("Year", year("Inspection_Date"))
#df.printSchema()

df.groupBy("Year").count().orderBy("Year").show()


root
 |-- INSPECTION_TYPE: string (nullable = true)
 |-- JOB_TICKET_OR_WORK_ORDER_ID: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- JOB_PROGRESS: string (nullable = true)
 |-- BBL: string (nullable = true)
 |-- BORO_CODE: string (nullable = true)
 |-- BLOCK: string (nullable = true)
 |-- LOT: string (nullable = true)
 |-- HOUSE_NUMBER: string (nullable = true)
 |-- STREET_NAME: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- X_COORD: string (nullable = true)
 |-- Y_COORD: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- Inspection_Date: timestamp (nullable = true)
 |-- RESULT: string (nullable = true)
 |-- APPROVED_DATE: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- COMMUNITY BOARD: string (nullable = true)
 |-- COUNCIL DISTRICT: string (nullable = true)
 |-- CENSUS TRACT: string (nullable = true)
 |-- BIN: string (null

In [41]:
from datetime import datetime
current_year = datetime.now().year
df = df.filter(
    (df["Year"] >= 2009) & (df["Year"] <= current_year)
)

In [42]:
df.groupBy("Year").count().orderBy("Year").show()

+----+------+
|Year| count|
+----+------+
|2009| 15056|
|2010|119530|
|2011|119345|
|2012|134383|
|2013|102754|
|2014|104736|
|2015|103745|
|2016|137217|
|2017|190500|
|2018|196431|
|2019|196306|
|2020| 55650|
|2021| 81534|
|2022|184757|
|2023|132503|
|2024| 47021|
|2025|   944|
+----+------+



In [25]:
#df.select("RESULT").distinct().show(truncate=False)

+------------------+
|RESULT            |
+------------------+
|Passed            |
|Failed for Other R|
|Monitoring visit  |
|Rat Activity      |
|Bait applied      |
|NULL              |
+------------------+



In [16]:
#df.groupBy("Borough", "Result").count().orderBy("Borough").show()


+---------+------------------+------+
|  Borough|            Result| count|
+---------+------------------+------+
|    Bronx|  Monitoring visit|    12|
|    Bronx|      Rat Activity| 86973|
|    Bronx|Failed for Other R| 51140|
|    Bronx|      Bait applied| 67130|
|    Bronx|            Passed|333611|
|    Bronx|              NULL|     9|
| Brooklyn|      Bait applied| 69313|
| Brooklyn|Failed for Other R| 40993|
| Brooklyn|              NULL|     2|
| Brooklyn|  Monitoring visit|    68|
| Brooklyn|      Rat Activity|100170|
| Brooklyn|            Passed|327200|
|Manhattan|            Passed|425771|
|Manhattan|  Monitoring visit|    26|
|Manhattan|      Bait applied| 87083|
|Manhattan|Failed for Other R| 21104|
|Manhattan|      Rat Activity| 97236|
|Manhattan|              NULL|    14|
|   Queens|  Monitoring visit|     7|
|   Queens|Failed for Other R| 39900|
+---------+------------------+------+
only showing top 20 rows



In [17]:
#df.groupBy("Zip_Code").count().orderBy('count', ascending=False).show(10)


+--------+-----+
|Zip_Code|count|
+--------+-----+
|   11216|82557|
|   11221|78581|
|   10458|74912|
|   10457|65642|
|   11237|60459|
|   10453|51032|
|   10456|50824|
|   11206|50379|
|   10027|41605|
|   10002|40644|
+--------+-----+
only showing top 10 rows



In [ ]:
#!pip install folium

In [23]:
#!pip install streamlit-folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 4.3 MB/s eta 0:00:00


In [43]:
df.coalesce(1).write.csv("rodent_cleaned_data.csv", header=True, mode="overwrite")

In [30]:
import folium
from folium.plugins import HeatMap
import streamlit as st
from streamlit_folium import st_folium

# Convert to pandas DataFrame for plotting
geo_df = df.select("Year","Latitude", "Longitude").dropna().toPandas()
years = sorted(geo_df['Year'].dropna().unique())
selected_year = st.selectbox("Select Year:", years)
df_year = geo_df[geo_df['Year'] == selected_year]
st.write(f"Total inspections for {selected_year}: {len(df_year)}")

df_year['lat_rounded'] = df_year['Latitude'].round(3)
df_year['lon_rounded'] = df_year['Longitude'].round(3)
heat_data = df_year.groupby(['lat_rounded', 'lon_rounded']).size().reset_index(name='count')
heat_list = heat_data[['lat_rounded', 'lon_rounded', 'count']].values.tolist()

# Create map
m = folium.Map(location=[40.867726534028, -73.887461100839], zoom_start=11)
#sample_geo_df = heat_list[['Latitude', 'Longitude']]#.head(5000)
HeatMap(heat_list).add_to(m)
st_data = st_folium(m, width=700, height=500)
#m


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#print(geo_df.shape)
#print(geo_df.head())

In [ ]:
#m.save("heatmap.html")

In [45]:
#!pip install streamlit pyngrok
from pyngrok import ngrok
#!ngrok config add-authtoken 2weUsgpHo13JMTYpHKuChzlPMcD_5QeiDZdEhqFcFrJnvn9AM
public_url = ngrok.connect("http://localhost:8501")
print(public_url)
!streamlit run untitled3.py

NgrokTunnel: "https://b0af-35-227-102-70.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.102.70:8501

^C


In [35]:
import pandas as pd
def load_data():
    return pd.read_csv("/content/drive/MyDrive/Colab Notebooks/rodent_cleaned_data.csv/part-00000-6b954623-2ac4-433f-bd49-111488b6ab95-c000.csv")
dfq = load_data()